## Setup

In [1]:
!pip install gradio

In [2]:
# Setup
import os
os.chdir('/content')
CODE_DIR = 'encoder4editing'

! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

!git clone https://github.com/omertov/encoder4editing.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

%cd
%cd /content/

! pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!git clone https://github.com/orpatashnik/StyleCLIP
!git clone https://github.com/esoyeon/Multilingual-StyleCLIP.git

# multilingual set up
!pip install ftfy regex tqdm gdown
!pip install transformers

os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp

%load_ext autoreload
%autoreload 2
%matplotlib inline

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.

In [3]:
# e4e setup

from gdown import download as drive_download
drive_download("https://drive.google.com/uc?id=1O8OLrVNOItOJoNGMyQ8G8YRTeTYEfs0P", "/content/encoder4editing/e4e_ffhq_encode.pt", quiet=False)
experiment_type = 'ffhq_encode'

os.chdir('/content/encoder4editing')

EXPERIMENT_ARGS = {
        "model_path": "e4e_ffhq_encode.pt"
    }
EXPERIMENT_ARGS['transform'] = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
resize_dims = (256, 256)

model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
# pprint.pprint(opts)  # Display full options used
# update the training options
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

Downloading...
From: https://drive.google.com/uc?id=1O8OLrVNOItOJoNGMyQ8G8YRTeTYEfs0P
To: /content/encoder4editing/e4e_ffhq_encode.pt
100%|██████████| 1.20G/1.20G [00:06<00:00, 189MB/s]


Loading e4e over the pSp framework from checkpoint: e4e_ffhq_encode.pt
Model successfully loaded!


## Image Alignment, Inversion

In [4]:
# Alignment
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

def run_alignment(image_path):
    import dlib
    from utils.alignment import align_face

    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    aligned_image = align_face(filepath=image_path, predictor=predictor)

    return aligned_image

--2023-12-26 17:57:53--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.1’

shape_predictor_68_ 100%[===================>]  61.07M  98.0MB/s    in 0.6s    

2023-12-26 17:57:53 (98.0 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.1’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


In [5]:
# Inversion

def run_on_batch(inputs, net):
    images, latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)
    if experiment_type == 'cars_encode':
        images = images[:, :, 32:224, :]
    return images, latents

## Global Direction

In [6]:
# Setup
os.chdir('/content/Multilingual-StyleCLIP')

sys.path.append(".")
sys.path.append("..")

# multilingual clip weights
!bash get-weights.sh

--2023-12-26 17:57:54--  https://www.dropbox.com/s/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl [following]
--2023-12-26 17:57:55--  https://www.dropbox.com/s/raw/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc63933eddaa2afb854f757bad21.dl.dropboxusercontent.com/cd/0/inline/CKJ4BIGEbZ3bD6NCzp4kOhyx2v1Dxwp8P4y6VxKfmmsSTJWqZg3XTY1LOtgsLONQkC2V_ChVvsm5fqwOYgn0EwM0pxGe9jYxb7pOYlzI4JJlg1JiKQAkFyGs_0HouPOoT60Z33iHGpGaxw-Tw--2LIYi/file# [following]
--2023-12-26 17:57:55--  https://uc63933eddaa2afb854f757bad21.dl.dropboxusercontent.com/cd/0/inline/CKJ4BIGEbZ3bD

In [7]:
# text prompt

imagenet_templates = [
'{}의 나쁜 사진.',
'{}의 조각상.',
'{} 보기 어려운 사진.',
'{}의 저해상도 사진.',
'{}의 렌더링.',
'{}의 숫자.',
'나쁜 {} 사진.',
'{}를 자른 사진.',
'{}의 문신.',
'자수 {}.',
'{} 보기 힘든 사진.',
'{}의 밝은 사진.',
'깨끗한 {}의 사진.',
'더러운 {}의 사진.',
'{}의 어두운 사진.',
'{} 그림.',
'내 {}의 사진.',
'플라스틱 {}',
'멋진 {}의 사진.',
'{}의 클로즈업 사진.',
'{}의 흑백 사진.',
'{}을 그린 그림.',
'{}를 그린 그림.',
'{}의 픽셀화된 사진.',
'{}의 조각상.',
'{}의 밝은 사진.',
'{}의 자른 사진.',
'플라스틱 {}.' ,
'더러운 {}의 사진.',
'{} JPEG 손상 사진.',
'{} 흐릿한 사진.',
'{} 사진.',
'{} 좋은 사진.',
'{} 렌더링.',
'비디오 게임 {}.',
'한 {} 사진.',
'{} 낙서.',
'{} 클로즈업 사진.',
'{} 사진.',
'종이접기 {}.',
'{} 비디오 게임.',
'{}의 스케치.',
'{}의 낙서.',
'{} 종이접기.',
'{}의 저해상도 사진.',
'장난감 {}.' ,
'{}을 변형한 것.',
'청렴한 {}의 사진.',
'큰 {}의 사진.',
'{}을 해석한 것.',
'멋진 {}의 사진.',
'이상한 {}의 사진.',
'{}의 흐릿한 사진.',
'만화 {}.' ,
'{} 작품.' ,
'{}의 스케치.',
'{} 자수.',
'{} 픽셀화된 사진.',
'{} itap.',
'{} JPEG 손상 사진.',
'좋은 {}의 사진.',
'{} 인형.' ,
'멋진 {}의 사진.',
'작은 {}의 사진.',
'이상한 {}의 사진.',
'만화 {}.' ,
'{} 예술.',
'{}을 그립니다.',
'큰 {}의 사진.',
'흑백 {} 사진.',
'인형 {}.' ,
'어두운 {} 사진.',
'내가 찍은 {} 사진.',
'{} 중 하나.',
'장난감 {}.' ,
'내 {} itap.',
'멋진 {} 사진.',
'작은 {}의 사진.',
'{}의 문신.'
]

In [12]:
# dataset

dataset_name='ffhq'

%cd
%cd /content/StyleCLIP/global_torch/

if not os.path.isfile('./model/'+dataset_name+'.pkl'):
        url='https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/'
        name='stylegan2-'+dataset_name+'-config-f.pkl'
        os.system('wget ' +url+name + '  -P  ./model/')
        os.system('mv ./model/'+name+' ./model/'+dataset_name+'.pkl')


# input prepare data

import clip
from manipulate import Manipulator
from StyleCLIP import GetDt,GetBoundary


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device,jit=False)

network_pkl='./model/'+dataset_name+'.pkl'
device = torch.device('cuda')
M=Manipulator()
M.device=device
G=M.LoadModel(network_pkl,device)
M.G=G
M.SetGParameters()
num_img=100_000
M.GenerateS(num_img=num_img)
M.GetCodeMS()
np.set_printoptions(suppress=True)

file_path='./npy/'+dataset_name+'/'
fs3=np.load(file_path+'fs3.npy')

/root
/content/StyleCLIP/global_torch
name:conv1_resolution_4 Resolution: 4, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_4 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_8 Resolution: 8, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_8 Resolution: 8, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_8 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_16 Resolution: 16, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_16 Resolution: 16, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_16 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_32 Resolution: 32, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_32 Resolution: 32, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_32 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_64 Resolution: 64, InC: 512, OutC:512, w_dim: 512
name:conv1_resolution_64 Resolution: 64, InC: 512, OutC:512, w_dim: 512
name:toRGB_resolution_64 InC: 512, OutC:3, w_dim: 512
name:conv0_resolution_128 Resolution: 128, InC: 512,

In [13]:
# Functions
os.chdir('/content/Multilingual-StyleCLIP/models')

import tensorflow as tf
import pickle
import copy
from MultilingualCLIP import multilingual_clip


def GetAlign(out, dt, model, preprocess):
    imgs = out
    imgs1 = imgs.reshape([-1]+list(imgs.shape[2:]))

    tmp = []
    for i in range(len(imgs1)):

        img = Image.fromarray(imgs1[i])
        image = preprocess(img).unsqueeze(0).to(device)
        tmp.append(image)

    image = torch.cat(tmp)

    with torch.no_grad():
        image_features = model.encode_image(image)
        image_features = image_features / \
            image_features.norm(dim=-1, keepdim=True)

    image_features1 = image_features.cpu().numpy()

    image_features1 = image_features1.reshape(list(imgs.shape[:2])+[512])

    fd = image_features1[:, 1:, :]-image_features1[:, :-1, :]

    fd1 = fd.reshape([-1, 512])
    fd2 = fd1/np.linalg.norm(fd1, axis=1)[:, None]

    tmp = np.dot(fd2, dt)
    m = tmp.mean()
    acc = np.sum(tmp > 0)/len(tmp)
    print(m, acc)
    return m, acc


def SplitS(ds_p, M, if_std):
    all_ds = []
    start = 0
    for i in M.mindexs:
        tmp = M.dlatents[i].shape[1]
        end = start+tmp
        tmp = ds_p[start:end]

        all_ds.append(tmp)
        start = end

    all_ds2 = []
    tmp_index = 0
    for i in range(len(M.s_names)):
        if (not 'RGB' in M.s_names[i]) and (not len(all_ds[tmp_index]) == 0):

            if if_std:
                tmp = all_ds[tmp_index]*M.code_std[i]
            else:
                tmp = all_ds[tmp_index]

            all_ds2.append(tmp)
            tmp_index += 1
        else:
            tmp = np.zeros(len(M.dlatents[i][0]))
            all_ds2.append(tmp)
    return all_ds2

## for multilingual  ##

def encode_text(txt):
    text_model = multilingual_clip.load_model('M-BERT-Base-ViT-B')
    return text_model(txt).cuda()

def zeroshot_classifier(classnames, templates, model):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in classnames:
            texts = [template.format(classname)
                     for template in templates]  # format with class

            # texts = clip.tokenize(texts).cuda() #tokenize (org)
            # class_embeddings = model.encode_text(texts) #embed with text encoder

            class_embeddings = encode_text(texts)

            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights


def GetDt(classnames, model, templates):
    text_features = zeroshot_classifier(classnames, templates, model).t()

    dt = text_features[0]-text_features[1]
    dt = dt.cpu().numpy()

    print(np.linalg.norm(dt))
    dt = dt/np.linalg.norm(dt)
    return dt

########################


def GetBoundary(fs3, dt, M, threshold):
    tmp = np.dot(fs3, dt)

    ds_imp = copy.copy(tmp)
    select = np.abs(tmp) < threshold
    num_c = np.sum(~select)

    ds_imp[select] = 0
    tmp = np.abs(ds_imp).max()
    ds_imp /= tmp

    boundary_tmp2 = SplitS(ds_imp, M, if_std=True)
    print('num of channels being manipulated:', num_c)
    return boundary_tmp2, num_c


def GetFs(file_path):
    fs = np.load(file_path+'single_channel.npy')
    tmp = np.linalg.norm(fs, axis=-1)
    fs1 = fs/tmp[:, :, :, None]
    fs2 = fs1[:, :, 1, :]-fs1[:, :, 0, :]  # 5*sigma - (-5)* sigma
    fs3 = fs2/np.linalg.norm(fs2, axis=-1)[:, :, None]
    fs3 = fs3.mean(axis=1)
    fs3 = fs3/np.linalg.norm(fs3, axis=-1)[:, None]
    return fs3


In [14]:
def save_image_and_get_path(input_image):
    if isinstance(input_image, np.ndarray):
        input_image = Image.fromarray(input_image) # numpy array to pil

    input_image = input_image.convert("RGB")

    image_path = '/content/uploaded_image.png'
    input_image.save(image_path)

    return image_path

## Gradio

In [18]:
# from models.global_directions.MapTS import GetFs,GetBoundary,GetDt
os.chdir('/content/Multilingual-StyleCLIP')

import gradio as gr

def multilingual_global(input_image, neutral, target, beta, alpha):
    # Alignment
    if isinstance(input_image, np.ndarray):
        input_image = Image.fromarray(input_image) # Convert numpy array to PIL Image

    input_image = input_image.convert("RGB")

    image_path = save_image_and_get_path(input_image)

    if experiment_type == "ffhq_encode" and 'shape_predictor_68_face_landmarks.dat' not in os.listdir():
        !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
        !bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

    if experiment_type == "ffhq_encode":
        aligned_image = run_alignment(image_path)
    else:
        aligned_image = input_image

    aligned_image = aligned_image.resize(resize_dims)

    # Inversion
    img_transforms = EXPERIMENT_ARGS['transform']
    transformed_image = img_transforms(aligned_image)

    # Run the image through the model
    with torch.no_grad():
        images, latents = run_on_batch(transformed_image.unsqueeze(0), net)
        result_image, latent = images[0], latents[0]
    torch.save(latents, '/content/latents.pt')

    # original image
    img_index = 0
    latents=torch.load('/content/latents.pt')
    dlatents_loaded=M.G.synthesis.W2S(latents)

    img_indexs=[img_index]
    dlatents_loaded=M.S2List(dlatents_loaded)

    dlatent_tmp=[tmp[img_indexs] for tmp in dlatents_loaded]

    M.num_images=len(img_indexs)

    M.alpha=[0]
    M.manipulate_layers=[0]
    codes,out=M.EditOneC(0,dlatent_tmp)
    original=Image.fromarray(out[0,0]).resize((512,512))
    M.manipulate_layers=None


    classnames = [target, neutral]
    dt = GetDt(classnames, model, imagenet_templates)

    M.alpha = [alpha]
    boundary, c = GetBoundary(fs3, dt, M, threshold=beta)
    codes = M.MSCode(dlatent_tmp, boundary)
    out = M.GenerateImg(codes)
    output_image = Image.fromarray(out[0,0])

    return output_image

# Define Gradio interface
iface = gr.Interface(
    fn=multilingual_global,
    inputs=[
        gr.Image(label="Input Image"),
        gr.Textbox(label="Neutral Text"),
        gr.Textbox(label="Target Text"),
        gr.Slider(minimum=0.08, maximum=0.3, step=0.01, label="Beta"),
        gr.Slider(minimum=-10, maximum=10, step=0.1, label="Alpha")
    ],
    outputs=gr.Image(label="Edited Image"),
    title="보정...해줘!",
    css=css
)

# Launch the app
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0c472d29a85c582df7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0.1409466
num of channels being manipulated: 123
0.34881043
num of channels being manipulated: 107
0.34881043
num of channels being manipulated: 107
0.1409466
num of channels being manipulated: 206
0.1409466
num of channels being manipulated: 206
0.1409466
num of channels being manipulated: 63
0.1409466
num of channels being manipulated: 33
0.1409466
num of channels being manipulated: 33
0.1409466
num of channels being manipulated: 16
0.14639546
num of channels being manipulated: 109
0.14639546
num of channels being manipulated: 109
0.14639546
num of channels being manipulated: 109
0.14639546
num of channels being manipulated: 109
0.1409466
num of channels being manipulated: 33
0.1409466
num of channels being manipulated: 33
0.1409466
num of channels being manipulated: 123
0.34881043
num of channels being manipulated: 65
0.07779812
num of channels being manipulated: 53
0.1409466
num of channels being manipulated: 123
0.1409466
num of channels being manipulated: 33
0.1409466
num of chan